# Baseline Evaluation for Helpdesk models

Inside the folder `<project_root>/runs/helpdesk` we have a list of folders named as `<percentage>%`, where `<percentage>` is the percentage of the dataset used for training the model.
In each of these folders we have a folder named as the best model found during the training phase based on the accuracy value.
Inside each of these folders we have the following files:
- `constraints_satisfaction_rate.csv`: a CSV file containing the constraints satisfaction rate for each of the test traces.
- `constraints_satisfactions.csv`: a CSV file containing the constraints satisfaction for each of the test traces.
- `predicted_traces.txt`: a TXT file containing the traces generated by the model for each of the test traces.
- `predictions.csv`: a CSV file containing the predictions step by step for each of the test traces.
- `results.json`: a JSON file containing the results of the evaluation of the model on the test set.

In [1]:
DATASET_NAME = "helpdesk"

In [2]:
from collections import namedtuple
import pathlib

project_root = pathlib.Path("../../..").parent.resolve()

Info = namedtuple("Info", ["model_args", "model_path", "results_path"])

models_path: dict[int, list[Info]] = {}

for dataset_percentage in range(20, 101, 20):
    checkpoints = [
        path
        for path in (project_root / "runs" / DATASET_NAME).rglob(
            f"{dataset_percentage}%/**/*.best_val_acc.pth"
        )
    ]
    results = [
        pathlib.Path(str(checkpoint).removesuffix(".pth")) / "results.json"
        for checkpoint in checkpoints
    ]
    args = [checkpoint.parent / "args.json" for checkpoint in checkpoints]
    models_path[dataset_percentage] = [
        Info(model_args=args, model_path=checkpoint, results_path=result)
        for args, checkpoint, result in zip(args, checkpoints, results)
    ]

## Comparison

In [3]:
import json
import pandas as pd

dataframes = {}

for percentage in models_path:
    dataframes[percentage] = pd.DataFrame(
        columns=[
            "lr",
            "dropout",
            "loss",
            "acc",
            "dld",
            "norm_dld",
            "constraints",
            "constraints_multiplier",
        ]
    )
    for info in models_path[percentage]:
        with open(info.model_args) as f:
            args = json.load(f)
        try:
            with open(info.results_path) as f:
                results = json.load(f)
                dataframes[percentage].loc[info.model_path.parent.name] = [
                    args["learning_rate"],
                    args["model"]["dropout"],
                    results["loss"],
                    results["acc"],
                    results["dld"],
                    results["norm_dld"],
                    args.get("constraints", None),
                    args.get("constraints_multiplier", None),
                ]
        except FileNotFoundError:
            print(f"Missing results for {info.model_path}")

/tmp/ipykernel_2370050/703172264.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframes[percentage].loc[info.model_path.parent.name] = [
/tmp/ipykernel_2370050/703172264.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframes[percentage].loc[info.model_path.parent.name] = [
/tmp/ipykernel_2370050/703172264.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA colu

In [4]:
for percentage in range(20, 101, 20):
    print("=" * 10 + f" {percentage}% " + "=" * 10)
    display(dataframes[percentage].sort_values("acc", ascending=False))
    print()

========== 20% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250303.2239.no_constraint,0.0010,0.0,0.119979,0.843621,2.846154,0.117233,[],NaN
20250303.2242.no_constraint,0.0001,0.2,0.113159,0.841564,2.923077,0.118740,[],NaN
20250303.2240.no_constraint,0.0001,0.0,0.114421,0.841564,2.923077,0.118740,[],NaN
20250303.2241.no_constraint,0.0010,0.2,0.116057,0.839506,2.923077,0.119636,[],NaN
20250304.0025.constraints,0.0001,0.0,0.241670,0.839506,2.961539,0.131561,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250304.0111.constraints,0.0001,0.2,0.231207,0.839506,3.000000,0.122863,"[Alternate Succession[Assign seriousness, Reso...",0.1
20250304.0022.constraints,0.0010,0.2,0.245293,0.837449,3.038461,0.134861,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250304.0018.constraints,0.0010,0.0,0.254159,0.835391,3.076923,0.137606,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250304.0057.constraints,0.0001,0.0,0.234063,0.833333,3.076923,0.137844,"[Alternate Succession[Assign seriousness, Reso...",0.1
20250304.0029.constraints,0.0001,0.2,0.239476,0.833333,3.076923,0.135167,"[Chain Succession[Resolve ticket, Closed]]",0.1



========== 40% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250303.2244.no_constraint,0.0010,0.2,0.151329,0.851249,2.854167,0.127160,[],NaN
20250303.2243.no_constraint,0.0001,0.0,0.122800,0.849077,2.895833,0.128483,[],NaN
20250303.2245.no_constraint,0.0001,0.2,0.122933,0.849077,2.895833,0.128483,[],NaN
20250304.0239.constraints,0.0001,0.0,0.235462,0.849077,2.895833,0.118674,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250303.2242.no_constraint,0.0010,0.0,0.141922,0.846906,2.937500,0.126477,[],NaN
20250304.0246.constraints,0.0001,0.2,0.225635,0.846906,2.937500,0.123492,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250304.0253.constraints,0.0010,0.0,0.212228,0.840391,3.062500,0.148805,"[Alternate Succession[Assign seriousness, Reso...",0.1
20250304.0337.constraints,0.0001,0.0,0.223734,0.837134,3.125000,0.137310,"[Alternate Succession[Assign seriousness, Reso...",0.1
20250304.0225.constraints,0.0010,0.0,0.207805,0.836048,3.104167,0.125194,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250304.0232.constraints,0.0010,0.2,0.211447,0.836048,3.145833,0.131878,"[Chain Succession[Resolve ticket, Closed]]",0.1



========== 60% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250303.2246.no_constraint,0.0010,0.0,0.081523,0.876027,2.593750,0.089562,[],NaN
20250303.2248.no_constraint,0.0010,0.2,0.081756,0.876027,2.593750,0.089562,[],NaN
20250303.2247.no_constraint,0.0001,0.0,0.079187,0.875280,2.609375,0.090120,[],NaN
20250303.2250.no_constraint,0.0001,0.2,0.079518,0.874533,2.625000,0.091682,[],NaN
20250304.0640.constraints,0.0001,0.2,0.184343,0.867812,2.765625,0.096351,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250304.0630.constraints,0.0001,0.0,0.178658,0.867065,2.781250,0.096839,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250304.0611.constraints,0.0010,0.0,0.164819,0.864824,2.828125,0.098027,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250304.0620.constraints,0.0010,0.2,0.163577,0.864824,2.828125,0.097970,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250304.0837.constraints,0.0001,0.2,0.172478,0.851382,3.078125,0.108063,"[Alternate Succession[Assign seriousness, Reso...",0.1
20250304.0723.constraints,0.0010,0.2,0.165745,0.851382,3.109375,0.105468,"[Alternate Succession[Assign seriousness, Reso...",0.1



========== 80% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250303.2256.no_constraint,0.0001,0.2,0.143383,0.854144,2.463636,0.134317,[],NaN
20250303.2253.no_constraint,0.0001,0.0,0.144588,0.853606,2.463636,0.134317,[],NaN
20250303.2251.no_constraint,0.0010,0.0,0.174217,0.852530,2.472727,0.139421,[],NaN
20250303.2254.no_constraint,0.0010,0.2,0.169270,0.850377,2.509091,0.137879,[],NaN
20250304.1359.constraints,0.0001,0.2,0.248293,0.848762,2.545455,0.139484,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250304.1341.constraints,0.0001,0.0,0.245656,0.845533,2.609091,0.141130,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250304.1305.constraints,0.0010,0.0,0.248112,0.844456,2.609091,0.141549,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250304.1324.constraints,0.0010,0.2,0.249004,0.844456,2.609091,0.142667,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250304.1521.constraints,0.0010,0.2,0.238531,0.825619,2.945455,0.164583,"[Alternate Succession[Assign seriousness, Reso...",0.1
20250304.1417.constraints,0.0010,0.0,0.248592,0.825619,2.927273,0.152534,"[Alternate Succession[Assign seriousness, Reso...",0.1



========== 100% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250303.2307.no_constraint,0.0001,0.0,0.122277,0.874892,2.206107,0.111077,[],NaN
20250303.2309.no_constraint,0.0010,0.2,0.140553,0.874892,2.206107,0.123032,[],NaN
20250303.2312.no_constraint,0.0001,0.2,0.119460,0.873593,2.221374,0.119007,[],NaN
20250303.2258.no_constraint,0.0010,0.0,0.146295,0.872727,2.244275,0.124350,[],NaN
20250304.2209.constraints,0.0010,0.2,0.216512,0.861905,2.435115,0.129282,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250304.2229.constraints,0.0001,0.0,0.223733,0.861905,2.412214,0.124752,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250304.2248.constraints,0.0001,0.2,0.223297,0.861472,2.419847,0.124990,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250304.2149.constraints,0.0010,0.0,0.222933,0.860173,2.458015,0.122279,"[Chain Succession[Resolve ticket, Closed]]",0.1
20250305.0009.constraints,0.0010,0.2,0.220478,0.855844,2.534351,0.129348,"[Alternate Succession[Assign seriousness, Reso...",0.1
20250304.2307.constraints,0.0010,0.0,0.233699,0.855844,2.541985,0.140355,"[Alternate Succession[Assign seriousness, Reso...",0.1


In [5]:
for percentage in range(20, 101, 20):
    print("=" * 10 + f" {percentage}% " + "=" * 10)
    display(dataframes[percentage].sort_values("acc", ascending=False).head(1))
    print()

========== 20% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250303.2239.no_constraint,0.001,0.0,0.119979,0.843621,2.846154,0.117233,[],NaN



========== 40% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250303.2244.no_constraint,0.001,0.2,0.151329,0.851249,2.854167,0.12716,[],NaN



========== 60% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250303.2246.no_constraint,0.001,0.0,0.081523,0.876027,2.59375,0.089562,[],NaN



========== 80% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250303.2256.no_constraint,0.0001,0.2,0.143383,0.854144,2.463636,0.134317,[],NaN



========== 100% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250303.2307.no_constraint,0.0001,0.0,0.122277,0.874892,2.206107,0.111077,[],NaN
